In [ ]:
from sqlalchemy import create_engine, text
import os
from sqlalchemy.engine import URL

url_object = URL.create(
    drivername="mysql+pymysql",
    username=os.getenv('user'),
    password=os.getenv('pw'),
    host=os.getenv('host'),
    port=os.getenv('port'),
    database=os.getenv('db_name')
)

engine = create_engine(url_object)

# SQL 명령어를 실행하기 위한 코드
sql_create_table = """
CREATE TABLE IF NOT EXISTS news_analysis (
    id INT AUTO_INCREMENT PRIMARY KEY,
    base_date DATETIME,
    title VARCHAR(500),
    content TEXT,
    ticker VARCHAR(10),
    pred_label TINYINT,
    pred_neg FLOAT,
    pred_neu FLOAT,
    pred_pos FLOAT,
    sentiment VARCHAR(20),
    INDEX idx_ticker_date (ticker, base_date)
);
"""

with engine.connect() as conn:
    conn.execute(text(sql_create_table))
    print("테이블 생성 완료!")

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine

# 2. CSV 파일들이 저장된 폴더 경로
folder_path = '../뉴스분류/뉴스분류'
file_list = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file_name in file_list:
    file_full_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_full_path)
    df['ticker'] = df['ticker'].astype(str).str.zfill(6)
    # 1. 날짜 변환 (기존 코드)
    df['base_date'] = pd.to_datetime(df['일자'], format='%Y%m%d', errors='coerce')
    
    # 2. ★ 컬럼명 매핑 (한글 -> 영어) ★
    # MySQL 테이블 정의(title, content 등)와 똑같이 맞춰줘야 합니다.
    df = df.rename(columns={
        '제목': 'title',
        '본문': 'content'
    })
    
    # 3. 필요 없는 기존 컬럼 삭제 (선택사항)
    # MySQL 테이블에 없는 '일자' 같은 컬럼이 남아있으면 에러가 날 수 있음
    df = df[['base_date', 'title', 'content', 'ticker', 'pred_label', 'prob_neg', 'prob_neu', 'prob_pos', 'sentiment']]
    
    # 4. DB에 저장
    try:
        df.to_sql(name='news_analysis', con=engine, if_exists='append', index=False)
        print(f"{file_name} 저장 성공!")
    except Exception as e:
        print(f"{file_name} 저장 실패: {e}")

print("모든 파일 업로드 완료.")

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM news_analysis"))
    count = result.scalar() # 결과값 하나만 가져오기
    print(f"현재 테이블에 저장된 총 행의 개수: {count}개")